In [34]:
import pandas as pd
df = pd.read_csv('C:/Users/User/Desktop/10/data-2/Week2_challenge_data_source(CSV).csv')

print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

<span style="font-size: 36px;">Task 1.1</span>

In [35]:

# Identify duplicated 'Bearer Id' values
duplicate_bearer_ids = df['Bearer Id'].value_counts()[df['Bearer Id'].value_counts() > 1]

duplicates_sample = df[df['Bearer Id'].isin(duplicate_bearer_ids.index)].head(10)
print("Sample of records with duplicated 'Bearer Id':\n", duplicates_sample)

duration_analysis = df[df['Bearer Id'].isin(duplicate_bearer_ids.index)]
duration_stats = duration_analysis.groupby('Bearer Id')['Dur. (ms)'].describe()
print("\nDuration analysis for each duplicated 'Bearer Id':\n", duration_stats)

aggregated_data = duration_analysis.groupby('Bearer Id').agg({
    'Dur. (ms)': 'sum',
    'Total UL (Bytes)': 'sum',
    'Total DL (Bytes)': 'sum',
}).reset_index()

print("\nAggregated data for each duplicated 'Bearer Id':\n", aggregated_data.head())


Sample of records with duplicated 'Bearer Id':
        Bearer Id            Start  Start ms             End  End ms  \
20  1.304243e+19   4/17/2019 0:35     363.0  4/25/2019 4:52   691.0   
30  1.304243e+19   4/18/2019 0:35     711.0  4/25/2019 4:52   691.0   
43  1.304243e+19   4/19/2019 2:55     809.0  4/25/2019 0:17   262.0   
50  1.304243e+19   4/20/2019 2:55     691.0  4/25/2019 0:17   262.0   
58  7.277826e+18  4/20/2019 14:08     136.0  4/25/2019 0:54    79.0   
62  1.304243e+19   4/21/2019 2:55      73.0  4/25/2019 0:17   262.0   
63  1.304243e+19   4/21/2019 3:04     521.0  4/25/2019 4:02   116.0   
72  7.277826e+18  4/21/2019 13:57     724.0  4/25/2019 0:54    79.0   
82  1.304243e+19   4/22/2019 0:35     293.0  4/25/2019 0:35    30.0   
84  1.304243e+19   4/22/2019 1:17     301.0  4/25/2019 1:17   938.0   

    Dur. (ms)          IMSI  MSISDN/Number          IMEI Last Location Name  \
20   706649.0  2.082010e+14   3.365875e+10  8.654080e+13            T29714B   
30   620248.

<span style="font-size: 30px;">Number of xDR sessions</span>

In [36]:

xdr_sessions_per_user = df.groupby('IMSI')['Bearer Id'].count().reset_index()
xdr_sessions_per_user.columns = ['IMSI', 'Number of xDR Sessions']

print("Number of xDR sessions per user:\n", xdr_sessions_per_user.head())


Number of xDR sessions per user:
            IMSI  Number of xDR Sessions
0  2.040471e+14                       1
1  2.040808e+14                       1
2  2.082001e+14                       1
3  2.082001e+14                       1
4  2.082001e+14                       2


<span style="font-size: 30px;">Session duration</span>

In [38]:
aggregated_data = df.groupby('Bearer Id')['Dur. (ms)'].sum().reset_index()

aggregated_data.columns = ['Bearer Id', 'Total Session Duration (ms)']

print("Aggregated session duration per user:\n", aggregated_data.head())

Aggregated session duration per user:
       Bearer Id  Total Session Duration (ms)
0  6.917538e+18                      24534.0
1  6.917538e+18                      21489.0
2  6.917538e+18                      27786.0
3  6.917538e+18                      15635.0
4  6.917538e+18                      24264.0


<span style="font-size: 30px;">The total download (DL) and upload (UL) data</span>

In [39]:
aggregated_data = df.groupby('Bearer Id').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
}).reset_index()

aggregated_data.columns = ['Bearer Id', 'Total Download Data (Bytes)', 'Total Upload Data (Bytes)']

print("Aggregated total download and upload data per user:\n", aggregated_data.head())

Aggregated total download and upload data per user:
       Bearer Id  Total Download Data (Bytes)  Total Upload Data (Bytes)
0  6.917538e+18                  500721999.0                 41704610.0
1  6.917538e+18                  709549265.0                 38693596.0
2  6.917538e+18                  854465860.0                 15548926.0
3  6.917538e+18                  825786510.0                 49605688.0
4  6.917538e+18                  624957763.0                 21983463.0
